In [1]:
import mesa
import seaborn as sns
import numpy as np
import pandas as pd
import requests
import json
# from agents import MoneyAgent
from utils.utils import citizenStates, raiderStates
from model import CityModel
from agents.CitizenAgent import CitizenAgent
from agents.RaiderAgent import RaiderAgent
from agents.WallAgent import WallAgent

In [2]:
def agent_portrayal(agent):
    size = 1
    # color = "tab:red"
    # if agent.wealth > 0:
    #     size = 10
    #     color = "tab:blue"

    if isinstance(agent,CitizenAgent):
        if agent.stateSystem is not None:
            if agent.stateSystem != "ok" and agent.stateSystem != "untracker":
                size = 4


        if agent.state == citizenStates.WALK:
            color = "tab:green"
        elif agent.state == citizenStates.SUSPICION:
            color = "gold"
        elif agent.state == citizenStates.PURSUED:
            color = "orange"
        elif agent.state == citizenStates.ASSAULTED:
            color = "orange"
            # size = 5
        else:
            print("Bad detections of instance!!!!")
    elif isinstance(agent,RaiderAgent):
        if agent.state == raiderStates.WALK:
            color = "lightcoral"
        elif agent.state == raiderStates.STALKIN:
            color = "salmon"
        elif agent.state == raiderStates.CHASING    :
            color = "red"
        elif agent.state == raiderStates.ASSAULTING:
            color = "red"
            size = 5
        elif agent.state == raiderStates.ESCAPE:
            color = "slateblue"
        else:
            print("Bad detections of instance!!!!")
    elif isinstance(agent,WallAgent):
        color = "lightgray"
        size = 0.01
        alpha = 0.7
        return {"size": size, "color": color,"alpha":alpha}
    else:
        print("Bad detections of instance!!!!")

    
    
    return {"size": size, "color": color}

In [3]:
model_params = {
    "num_citizen": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of citizen:",
        "min": 1,
        "max": 300,
        "step": 1,
    },
    "num_raider": 15,
    "width": 70, # If pathMap is not None, this value doesn't matter
    "height": 70, # If pathMap is not None, this value doesn't matter
    "distance_to_commute_citizen": 30,
    "distance_to_commute_raider": 30,
    "distance_field_of_view_citizen":12,
    "distance_field_of_view_raider":12,
    "speed_citizen":1,
    "speed_raider":1,
    "countDownSuspicion":4,
    "countDownChase":6,
    "minCitizenToStalk":2, # Itera
    # "pathMap":"mapsImage/testImg.png",
    "pathMap":"mapsImage/mapReadyFinal.png",
    "thresholdMap":244,
    "use_system":False
}

In [4]:
from mesa.experimental import JupyterViz

page = JupyterViz(
    CityModel,
    model_params,
    # measures=["objetivosCompletados","persecucionesFallidas",
    #           "asechoFallido","asechoDetectado",
    #           "persecucionDetectada","victimasSorprendidas"],
    measures=["asaltosRealizados"],
    # measures=[],
    name="City Model",
    agent_portrayal=agent_portrayal,
)
# This is required to render the visualization in the Jupyter notebook
page

0
Map size:  (388, 724)
0
Map size:  (388, 724)


Cannot show ipywidgets in text

In [5]:
import os
class Lhs_generator():
    def __init__(self,iterations,model_params,model_results,outcome):
        self.model_params = model_params
        self.model_results = pd.DataFrame(model_results)
        self.model_outcome = outcome
        self.iterations = iterations
    def lhsMatrixGenerator(self):
        lhsVars = [variable for variable, valor in model_params_batch.items()
                   if self.is_iterable(valor) and isinstance(valor, str) is False]
        groupIteration = self.model_results.groupby(['RunId', 'iteration']).mean(numeric_only=True).reset_index().iloc[:,:11]
        groupIteration = groupIteration.groupby(lambda x: x % iters).mean().reset_index()
        # groupIteration = self.model_results.groupby('iteration').mean(numeric_only=True).reset_index()
        lhsdata = groupIteration[lhsVars].T
        lhsdata.to_csv("output/lhsmatrix",index=False, header=False, sep='\t')
        with open("output/lhsmatrix", 'r+') as archivo:
            contenido_actual = archivo.read()
            archivo.seek(0, 0)
            archivo.write(f"{lhsdata.shape[0]}\t{lhsdata.shape[1]}\t2\t{151}" + '\n')
            archivo.write(contenido_actual)
        # print(lhsdata)
    def lhsFilesValues(self):
        archivos_a_eliminar = [archivo for archivo in os.listdir("output") if archivo.isdigit()]

        for archivo in archivos_a_eliminar:
            ruta_archivo = os.path.join("output", archivo)
            os.remove(ruta_archivo)

        # print(self.model_results[(self.model_results.RunId == 0)][self.model_outcome])
        list_group_by_same_parameters = []
        iters = self.iterations
        for nombre_grupo, grupo in self.model_results.groupby(["RunId"]):
            # Crear un nuevo DataFrame para el grupo actual
            if int(nombre_grupo)<iters:
                list_group_by_same_parameters.append([nombre_grupo,pd.DataFrame(grupo)[self.model_outcome].reset_index()/iters])
            else:
                # print("Nombre de grupo: ", nombre_grupo)
                # print(pd.DataFrame(grupo)["objetivosCompletados"].reset_index()/iters+list_group_by_same_parameters[int(nombre_grupo)%iters][1])
                list_group_by_same_parameters[int(nombre_grupo)%iters][1]+= pd.DataFrame(grupo)[self.model_outcome].reset_index()/iters
        
        for combinacion, step_values in list_group_by_same_parameters:
            nombre_archivo = f'output/{int(combinacion)+1:04}'
            temDataFrame = pd.DataFrame(step_values).drop(columns=["index"])
            temDataFrame.to_csv(nombre_archivo,sep='\t',header=False)
            # df_grupo.to_csv(nombre_archivo, index=False)
            print(temDataFrame)
            # Imprimir mensaje de éxito
            print(f'Se guardó la combinacion {int(combinacion)+1:04} en el archivo {nombre_archivo}.')
    
    def lhsOutcomeGenerator(self):
        pass # Not priority
    def is_iterable(self, obj):
        try:
            iter(obj)
            return True
        except TypeError:
            return False
# outcome = ["objetivosCompletados","persecucionesFallidas",
#            "asechoFallido","asechoDetectado",
#            "persecucionDetectada","victimasSorprendidas"]
# lhs_generator = Lhs_generator(iters,model_params_batch,results,outcome)
# # lhs_generator.lhsMatrixGenerator()
# # lhs_generator.lhsFilesValues()

In [11]:
def lhsDataReadFromFile(file_name = 'output/lhsmatrix'):
        with open(file_name, 'r') as archivo:
            lineas = archivo.readlines()

        # Ignorar la primera línea
        datos = [linea.strip().split() for linea in lineas[1:]]

        # Definir nombres de columnas
        columnas = [
            "num_citizen",
            "num_raider",
            "distance_to_commute_citizen",
            "distance_to_commute_raider",
            "distance_field_of_view_citizen",
            "distance_field_of_view_raider",
            "speed_citizen",
            "speed_raider",
            "countDownSuspicion",
            "countDownChase",
            "minCitizenToStalk"
        ]

        # Crear DataFrame transpuesto
        # df = pd.DataFrame(datos, columns=columnas).transpose()
        df = pd.DataFrame(datos).transpose()
        df.columns = columnas
        df = df.applymap(lambda x: round(float(x)))
        return df
lhs_vals = lhsDataReadFromFile()
lhs_vals

,num_citizen,num_raider,distance_to_commute_citizen,distance_to_commute_raider,distance_field_of_view_citizen,distance_field_of_view_raider,speed_citizen,speed_raider,countDownSuspicion,countDownChase,minCitizenToStalk
0,38,11,100,140,6,6,2,1,1,1,1
1,42,9,160,160,9,9,2,2,9,6,4
2,32,6,140,100,14,11,1,2,6,4,6
3,48,14,120,120,11,14,3,3,4,9,9


In [10]:
import os
import pickle

resultados_dict = {}
checkpoint_interval = 2
# Configurar LHS RUN
for i, parameters in lhs_vals.iterrows():
    # print(parameters['num_citizen'])
    # Crear el modelo con los parámetros específicos
    model = CityModel(
        int(parameters['num_citizen']),
        int(parameters['num_raider']),
        70,
        70,
        int(parameters['distance_to_commute_citizen']),
        int(parameters['distance_to_commute_raider']),
        int(parameters['distance_field_of_view_citizen']),
        int(parameters['distance_field_of_view_raider']),
        int(parameters['speed_citizen']),
        int(parameters['speed_raider']),
        int(parameters['countDownSuspicion']),
        int(parameters['countDownChase']),
        int(parameters['minCitizenToStalk']),
        pathMap="mapsImage/mapReadyFinal.png"
    )

    # Ejecutar el modelo durante 30 pasos
    for step in range(2):
        model.step()

    # Obtener métricas y guardar en archivo
    metrics = model.datacollector.get_model_vars_dataframe()
    key = f'{i+1:04}'  # Nombre de archivo
    resultados_dict[key] = metrics
    if (i + 1) % checkpoint_interval == 0:
        checkpoint_filename = f'outputDump/checkpoint_{i+1}.pkl'
        with open(checkpoint_filename, 'wb') as f_checkpoint:
            pickle.dump(resultados_dict, f_checkpoint)
    # output_folder = 'output'
    # os.makedirs(output_folder, exist_ok=True)
    # output_filename = os.path.join(output_folder, f'{i+1:04}.txt')
    metrics.to_csv('output/'+f'{i+1:04}', sep='\t', header=False, index=True,float_format='%.6e')

with open('outputDump/resultados_dict_total.pkl', 'wb') as f:
    pickle.dump(resultados_dict, f)


0
Map size:  (388, 724)
0
Map size:  (388, 724)
0
Map size:  (388, 724)
0
Map size:  (388, 724)


In [14]:
import pickle

# Cargar el diccionario desde el archivo
with open('outputDump/resultados_dict_total.pkl', 'rb') as f:
    resultados_dict_recuperado = pickle.load(f)


df_0001 = resultados_dict_recuperado['0001']
df_0001

,asaltosRealizados
0,0.01
1,0.01


In [13]:
import os
import pickle
from concurrent.futures import ProcessPoolExecutor , as_completed

resultados_dict = {}
checkpoint_interval = 2

# Función para ejecutar el modelo y obtener métricas
def ejecutar_modelo(parameters):
    # Crear el modelo con los parámetros específicos
    model = CityModel(
        int(parameters['num_citizen']),
        int(parameters['num_raider']),
        70,
        70,
        int(parameters['distance_to_commute_citizen']),
        int(parameters['distance_to_commute_raider']),
        int(parameters['distance_field_of_view_citizen']),
        int(parameters['distance_field_of_view_raider']),
        int(parameters['speed_citizen']),
        int(parameters['speed_raider']),
        int(parameters['countDownSuspicion']),
        int(parameters['countDownChase']),
        int(parameters['minCitizenToStalk']),
        pathMap="mapsImage/mapReadyFinal.png"
    )

    # Ejecutar el modelo durante 30 pasos
    for step in range(2):
        model.step()

    # Obtener métricas
    metrics = model.datacollector.get_model_vars_dataframe()

    return metrics

# Configurar LHS RUN en paralelo
with ProcessPoolExecutor() as executor:
    futures = {executor.submit(ejecutar_modelo, parameters): i for i, parameters in lhs_vals.iterrows()}

    for future in as_completed(futures):
        i = futures[future]
        try:
            metrics = future.result()
            key = f'{i+1:04}'  # Nombre de archivo
            resultados_dict[key] = metrics
            if (i + 1) % checkpoint_interval == 0:
                checkpoint_filename = f'outputDump/checkpoint_{i+1}.pkl'
                with open(checkpoint_filename, 'wb') as f_checkpoint:
                    pickle.dump(resultados_dict, f_checkpoint)
            metrics.to_csv('output/'+f'{i+1:04}', sep='\t', header=False, index=True, float_format='%.6e')
        except Exception as e:
            print(f"Error en la iteración {i+1}: {e}")

with open('outputDump/resultados_dict_total.pkl', 'wb') as f:
    pickle.dump(resultados_dict, f)


00
0

0
Map size:  (388, 724)
Map size:  (388, 724)
Map size: Map size:   (388, 724)(388, 724)



In [ ]:
model_params_batch = {
    "num_citizen": 300, # range(200,500,200),
    "num_raider": 10, # range(4,20,8),
    "width": 70, # If pathMap is not None, this value doesn't matter
    "height": 70, # If pathMap is not None, this value doesn't matter
    "distance_to_commute_citizen": 50, # range(25,35,6),
    "distance_to_commute_raider": 50, # range(25,35,6),
    "distance_field_of_view_citizen":[10,16], # range(10,20,6),
    "distance_field_of_view_raider":[10,16], # range(10,20,6),
    "speed_citizen":1, #range(1,3,1),
    "speed_raider":1, # range(1,3,1),
    "countDownSuspicion":4, # range(3,6,2),
    "countDownChase":6, # range(3,6,2),
    # "minCitizenToStalk":range(2,5,2), # Itera
    "minCitizenToStalk":2, # Itera
    # "pathMap":"testImg.png",
    "pathMap":"mapsImage/mapReadyFinal.png",
    "thresholdMap":244
}
iters = 4

results = mesa.batch_run(
    CityModel,
    parameters=model_params_batch,
    iterations=iters,
    max_steps=3,
    number_processes=7,
    data_collection_period=1,
    display_progress=True,
)

In [ ]:
# results_df = pd.DataFrame(results)
# print(results_df.keys())

In [ ]:
# results_df[(results_df.RunId == 0) | (results_df.RunId == 1)][["RunId"	,"iteration"	,"Step" ,"asechoFallido","distance_field_of_view_citizen",	"distance_field_of_view_raider"]]

In [ ]:
# # varToGroup = results_df.columns.to_list()
# # varToGroup.remove("iteration")
# # varToGroup.remove("RunId")
# # testTemp = results_df.groupby(['RunId', 'iteration']).mean(numeric_only=True).reset_index().iloc[:,:11]
# testTemp = results_df.groupby(['RunId']).mean(numeric_only=True).reset_index().iloc[:,:11]
# # testTemp.groupby(lambda x: x % 4).mean().reset_index()
# testTemp

In [ ]:
# list_group_by_same_parameters = []
# iters = 4
# for nombre_grupo, grupo in results_df.groupby(["RunId"]):
#     # Crear un nuevo DataFrame para el grupo actual
#     if int(nombre_grupo)<iters:
#         list_group_by_same_parameters.append([nombre_grupo,pd.DataFrame(grupo)["asechoFallido"].reset_index()/iters])
#     else:
#         # print("Nombre de grupo: ", nombre_grupo)
#         # print(pd.DataFrame(grupo)["objetivosCompletados"].reset_index()/iters+list_group_by_same_parameters[int(nombre_grupo)%iters][1])
#         list_group_by_same_parameters[int(nombre_grupo)%iters][1]+= pd.DataFrame(grupo)["asechoFallido"].reset_index()/iters

#     # df_grupo = pd.DataFrame(grupo)[["RunId"	,"iteration"	,"Step" ,"asechoFallido","distance_field_of_view_citizen",	"distance_field_of_view_raider"]]

# # Guardar el DataFrame en un archivo con un nombre único
# for combinacion, step_values in list_group_by_same_parameters:
#     nombre_archivo = f'{combinacion}.csv'
#     # df_grupo.to_csv(nombre_archivo, index=False)
#     print(pd.DataFrame(step_values))
#     # Imprimir mensaje de éxito
#     print(f'Se guardó la combinacion {combinacion} en el archivo {nombre_archivo}.')

In [ ]:
# import os
# archivos_a_eliminar = [archivo for archivo in os.listdir("output") if archivo.isdigit()]

# for archivo in archivos_a_eliminar:
#     ruta_archivo = os.path.join("output", archivo)
#     os.remove(ruta_archivo)